# BERT NE recognition FACILITY

## 1)  Preprocess / format data

In [1]:
from __future__ import absolute_import, division, print_function

In [2]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re
import keras
import nltk
import spacy

Using TensorFlow backend.


In [3]:
from nltk import ne_chunk, pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize, PunktSentenceTokenizer
from nltk.corpus import stopwords
import gensim.models.word2vec as w2v
from gensim import logging
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as  pd
import seaborn as sns
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import json
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import MWETokenizer

In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [6]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/chantana/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## 2) load keyword: LOC from text

In [7]:
word_files = ['./keywords/list_hotel.txt']
word_list_name = []
for word_file in word_files:
    with open(word_file) as f:
        lines = f.readlines()
    word_list2 = [x.strip() for x in lines] 
    
    word_list_name.extend(word_list2)
word_list_name[0:5]

word_hotels = ['./keywords/list_location.txt']
lists_location = []
for word_hotel in word_hotels:
    with open(word_hotel) as f:
        lines = f.readlines()
    lists_l= [x.strip() for x in lines] 
    
    lists_location.extend(lists_l)
lists_location[0:5]

['99/89 Moo 6',
 'VIPHAVADEE RANDSIT RD',
 '111/1 Soi Ngam Wong Wan 47 Yaek 1',
 '310/72 Soi Phaholyothin 67/1',
 '3/2899 Soi Yak']

In [8]:
#remove duplicates
def ordered_set(in_list):
    out_list = []
    added = set()
    for val in in_list:
        if not val in added:
            out_list.append(val)
            added.add(val)
    return out_list

In [9]:
word_list_name = ordered_set(word_list_name)
word_location= ordered_set(lists_location)

In [10]:
mw_list = []

     
for x in word_list_name:
    ws = x.split() 
    if len(ws) > 1:
        tws = tuple(ws)
        mw_list.append(tws)
        

for x in word_location:
    ws = x.split() 
    if len(ws) > 1:
        tws = tuple(ws)
        mw_list.append(tws)
        

            
mw_list = ordered_set(mw_list)
mw_list[0:5]

[('Petpimarn', 'Boutique', 'Resort'),
 ('Airport', 'Suite', 'Bangkok', '(Don', 'Muang', 'Airport)'),
 ('The', 'Riche', 'Residence'),
 ('Regent', 'Home1', 'at', 'Donmuang'),
 ('Charoenpong', 'Apartment')]

In [11]:
my_idx = {}
for i,w in enumerate(mw_list):
    my_idx[" ".join(w)] = i
print(len(my_idx))

35803


In [12]:
mwe = MWETokenizer(mw_list,separator=' ')

In [13]:
reviews = pd.read_csv("./keywords/location.csv")

In [14]:
reviews.shape

(13019, 1)

In [15]:
reviews.head()

location
0  Staying at @ Home Executive Apartment is a goo...
1  Staying at @ Oasis Resort is a good choice whe...
2  Staying at @ T Boutique Hotel is a good choice...
3  Staying at AC Resort is a good choice when you...
4  Staying at Acqua Condominium is a good choice ...

In [16]:
reviews = reviews.dropna()
reviews = reviews.reset_index(drop=True)

In [17]:
def spans(txt):
    tokens=mwe.tokenize(word_tokenize(txt))
    offset = 0
    for token in tokens:
        offset = txt.find(token, offset)
        yield token, offset, offset+len(token)
        offset += len(token)

In [78]:
from tqdm import tqdm_notebook as tqdm
all_item = []

for i in tqdm(range(len(reviews))):
    word_ls = []
    #print("sentence:",i+1)
    
    for token in spans(reviews.location[i]):
        #print(token)
        #assert token[0]==reviews.location[i][token[1]:token[2]]
        my_tuple = token[0]
        #print("###",token[0].lower())
        
        #my_tuples = ' , '.join(map(str, my_tuple))
        
         
        if token[0] in word_location:
            #word_ls.append(my_tuple)
            subwords = token[0].split()
            pos_list = [nltk.pos_tag([w]) for w in subwords]
            tag_list = ['I-LOC']*len(pos_list)
            tag_list[0] = 'B-LOC' 
            
            
            for s,p,t in zip(subwords,pos_list,tag_list):           
                if type(p) == list:
                    p = p[0][1]
                    #print('list')
                new_item = dict({'Sentence #': i+1, 'Tag' : t, 'Word': s,'POS': p})
                all_item.append(new_item)
                 
                
            #lis_lo = nltk.pos_tag(word_ls),LOC
            #print(' , '.join(map(str, lis_lo)))
 
        elif token[0] in word_list_name:
            
            subwords = token[0].split()
            pos_list = [nltk.pos_tag([w]) for w in subwords]
            tag_list = ['I-ORG']*len(pos_list)
            tag_list[0] = 'B-ORG' 
            
            #print(subwords)
            for s,p,t in zip(subwords,pos_list,tag_list):           
                new_item = dict({'Sentence #': i+1, 'Tag' : t, 'Word': s,'POS': p[0][1]})
                all_item.append(new_item)
                #print(new_item)
            #word_ls.append(my_tuple)
            #print("found LOC")
            #lis_lo = nltk.pos_tag(word_ls),"LOC"
            #print(' , '.join(map(str, lis_lo)))
                 

        else:
            #print(type(my_tuple))
            my_pos = nltk.pos_tag([my_tuple])[0][1]
            new_item = dict({'Sentence #': i+1, 'Tag' : 'O', 'Word': my_tuple.lower(),'POS': my_pos})
            all_item.append(new_item)
            
            #if i+1 == 5177:
                #print(new_item)
             
            #print(nltk.pos_tag([my_tuple]),',','O')
        #print(new_item)
        #if not(my_pos == '.' or  my_pos == ',' or my_pos == ':' or my_pos == '(' or my_pos == ')') :
            
            #all_item.append(new_item) 

        

        
    

          

## 3) save preproc data

In [79]:
file_csv = 'data-loc-bert.csv'
with open(file_csv, 'w') as csv_file:
    csv_file.write('Sentence #*Word*POS*Tag\n')
    for item in all_item:
        
        #print(item['POS'])    
        csv_file.write(str(item['Sentence #'])+'*'+item['Word']+'*'+item['POS']+'*'+item['Tag']+'\n')
csv_file.close()

## 4) Load preprocessed data

In [18]:
import pandas as pd
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
#load data
# explore data
file_csv = 'data-loc-bert.csv'
data = pd.read_csv(file_csv, sep='*',encoding="utf-8").fillna(method="ffill")

data.head(25)
 


Sentence #       Word  POS    Tag
0            1    staying  VBG      O
1            1         at   IN      O
2            1          @   NN  B-ORG
3            1       Home   NN  I-ORG
4            1  Executive   NN  I-ORG
5            1  Apartment   NN  I-ORG
6            1         is  VBZ      O
7            1          a   DT      O
8            1       good   JJ      O
9            1     choice   NN      O
10           1       when  WRB      O
11           1        you  PRP      O
12           1        are  VBP      O
13           1   visiting  VBG      O
14           1    Central   JJ  B-LOC
15           1    Pattaya   NN  I-LOC
16           1          .    .      O
17           2    staying  VBG      O
18           2         at   IN      O
19           2          @   NN  B-ORG
20           2      Oasis   NN  I-ORG
21           2     Resort   NN  I-ORG
22           2         is  VBZ      O
23           2          a   DT      O
24           2       good   JJ      O

### 5) BERT trained-formating

In [19]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [20]:
 
#concat sentence
getter = SentenceGetter(data)

In [21]:
##word_list = [ [s[0] for s in sent] for sent in getter.sentences] 

In [22]:
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

u'staying at @ Home Executive Apartment is a good choice when you are visiting Central Pattaya .'

In [23]:
print(len(sentences))

13019


In [24]:
labels = [[s[2] for s in sent] for sent in getter.sentences]
print(len(labels[0]))

17


In [25]:
print(len(labels))

13019


In [26]:
tags_vals = list(set(data["Tag"].values))
tag2idx = {t: i for i, t in enumerate(tags_vals)}
idx2tag = {i: t for i, t in enumerate(tags_vals) }

# Training

In [27]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [30]:
MAX_LEN = 128
bs = 8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0) 
torch.cuda.set_device(1)


In [31]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)

2019-03-18 17:33:49,200 : INFO : loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-vocab.txt from cache at /home/chantana/.pytorch_pretrained_bert/9b3c03a36e83b13d5ba95ac965c9f9074a99e14340c523ab405703179e79fc46.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


mwe = MWETokenizer(mw_list,separator=' ')

tokenized_texts = [mwe.tokenize(word_tokenize(sent)) for sent in sentences]


print(tokenized_texts[0],len(tokenized_texts[0]))

In [32]:
tokenized_texts = [ tokenizer.tokenize(sent)   for sent in sentences]
print(tokenized_texts[0])

[u'staying', u'at', u'@', u'home', u'executive', u'apartment', u'is', u'a', u'good', u'choice', u'when', u'you', u'are', u'visiting', u'central', u'pat', u'##ta', u'##ya', u'.']


In [33]:
sentences[0]

u'staying at @ Home Executive Apartment is a good choice when you are visiting Central Pattaya .'

In [34]:
print(tokenized_texts[0])

[u'staying', u'at', u'@', u'home', u'executive', u'apartment', u'is', u'a', u'good', u'choice', u'when', u'you', u'are', u'visiting', u'central', u'pat', u'##ta', u'##ya', u'.']


In [41]:
def convert_word2idx (text_list):
    
    values = []
    #print(text_list)
    start = len(tokenizer.vocab)
    for word in text_list:
        
        if tokenizer.vocab.has_key(word):
            values.append(tokenizer.vocab[word])
        else:
            #print(word,"not found")
            tokenizer.vocab[word] = start
            values.append(tokenizer.vocab[word])
            start = start + 1
        
            
    return values


In [42]:
len(tokenizer.vocab)

30522

In [43]:
tokenizer_save =  tokenizer


In [44]:
tokenizer.vocab 

OrderedDict([(u'[PAD]', 0),
             (u'[unused0]', 1),
             (u'[unused1]', 2),
             (u'[unused2]', 3),
             (u'[unused3]', 4),
             (u'[unused4]', 5),
             (u'[unused5]', 6),
             (u'[unused6]', 7),
             (u'[unused7]', 8),
             (u'[unused8]', 9),
             (u'[unused9]', 10),
             (u'[unused10]', 11),
             (u'[unused11]', 12),
             (u'[unused12]', 13),
             (u'[unused13]', 14),
             (u'[unused14]', 15),
             (u'[unused15]', 16),
             (u'[unused16]', 17),
             (u'[unused17]', 18),
             (u'[unused18]', 19),
             (u'[unused19]', 20),
             (u'[unused20]', 21),
             (u'[unused21]', 22),
             (u'[unused22]', 23),
             (u'[unused23]', 24),
             (u'[unused24]', 25),
             (u'[unused25]', 26),
             (u'[unused26]', 27),
             (u'[unused27]', 28),
             (u'[unused28]', 29),
     

In [45]:
print(mw_list[:5])
for i in mw_list:
    if " ".join(i) == '@ Home Executive Apartment':
        print('found')

[('Petpimarn', 'Boutique', 'Resort'), ('Airport', 'Suite', 'Bangkok', '(Don', 'Muang', 'Airport)'), ('The', 'Riche', 'Residence'), ('Regent', 'Home1', 'at', 'Donmuang'), ('Charoenpong', 'Apartment')]
found


In [102]:
def append_vocab(vocab_tuple_list):
    start = len(tokenizer.vocab)
    
    for v in vocab_tuple_list:
        ww = " ".join(v)
        if ww not in tokenizer.vocab:
            #print(v)
            tokenizer.vocab[ww] = start
            print(ww)
            start += 1
    return tokenizer.vocab


In [35]:
tokens_ids = [  tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts]


In [36]:
 print(len(tokens_ids[0]),len(labels[0]))
 

19 17


In [37]:
#input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
input_ids = pad_sequences(tokens_ids,
                          maxlen=MAX_LEN, dtype="int64", truncating="post", padding="post")
#input_ids = pad_sequences([convert_word2idx(txt) for txt in tokenized_texts],
                          #maxlen=MAX_LEN, dtype="int64", truncating="post", padding="post")

In [49]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="int64", truncating="post", padding="post")


In [97]:
print(MAX_LEN)
for i in tokens_ids:
    if len(i) > MAX_LEN:
        #print(tokens_ids)
        print("more")

128


In [36]:
t_list = [[tag2idx.get(l) for l in lab] for lab in labels]
len(t_list[0])

17

In [52]:
print(input_ids[0])

[6595 2012 1030 2188 3237 4545 2003 1037 2204 3601 2043 2017 2024 5873
 2430 6986 2696 3148 1012    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [38]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="int64", truncating="post")

In [39]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [40]:
#split train test
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.3)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.3)

In [41]:
#change to torhc tensor
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [42]:
print(tr_inputs.shape)
print(tr_masks.shape)
print(tr_tags.shape)

torch.Size([9113, 128])
torch.Size([9113, 128])
torch.Size([9113, 128])


In [43]:
print(val_inputs.shape)
print(val_masks.shape)
print(val_tags.shape)

torch.Size([3906, 128])
torch.Size([3906, 128])
torch.Size([3906, 128])


In [44]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [45]:
print(len(tag2idx))

5


In [46]:
#torch.cuda.empty_cache()

In [47]:
model = BertForTokenClassification.from_pretrained(u"bert-large-uncased", num_labels=len(tag2idx))

2019-03-18 17:34:17,583 : INFO : loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased.tar.gz from cache at /home/chantana/.pytorch_pretrained_bert/214d4777e8e3eb234563136cd3a49f6bc34131de836848454373fa43f10adc5e.abfbb80ee795a608acbf35c7bf2d2d58574df3887cdd94b355fc67e03fddba05
2019-03-18 17:34:17,587 : INFO : extracting archive file /home/chantana/.pytorch_pretrained_bert/214d4777e8e3eb234563136cd3a49f6bc34131de836848454373fa43f10adc5e.abfbb80ee795a608acbf35c7bf2d2d58574df3887cdd94b355fc67e03fddba05 to temp dir /tmp/tmpVp2vL3
2019-03-18 17:34:28,339 : INFO : Model config {
  "attention_probs_dropout_prob": 0.1, 
  "hidden_act": "gelu", 
  "hidden_dropout_prob": 0.1, 
  "hidden_size": 1024, 
  "initializer_range": 0.02, 
  "intermediate_size": 4096, 
  "max_position_embeddings": 512, 
  "num_attention_heads": 16, 
  "num_hidden_layers": 24, 
  "type_vocab_size": 2, 
  "vocab_size": 30522
}

2019-03-18 17:34:39,056 : INFO : Weights of BertForTokenCla

In [48]:
model.cuda();

In [49]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [50]:
#Fine tune BERT
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat) 

In [51]:
epochs = 20
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Train loss: 0.462766189677
Validation loss: 0.359476895969
Validation Accuracy: 0.86926444274


Epoch:   5%|▌         | 1/20 [09:45<3:05:22, 585.38s/it]

F1-Score: 0.27090575817
Train loss: 0.303193381141
Validation loss: 0.26072880903
Validation Accuracy: 0.903072677019


Epoch:  10%|█         | 2/20 [19:41<2:56:37, 588.72s/it]

...

F1-Score: 0.727872237673
Train loss: 0.0601798474985
Validation loss: 0.10534445975
Validation Accuracy: 0.958489104039


Epoch:  95%|█████████▌| 19/20 [3:09:58<10:01, 601.71s/it]

F1-Score: 0.737606516086
Train loss: 0.0601331101471
Validation loss: 0.153146133314
Validation Accuracy: 0.955523469453


Epoch: 100%|██████████| 20/20 [3:20:01<00:00, 602.23s/it]

F1-Score: 0.715634906296


In [52]:
#evaluate model
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
print(len(valid_dataloader))
for batch in tqdm(valid_dataloader):
    #print(len(batch))
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

  0%|          | 0/489 [00:00<?, ?it/s]

489


100%|██████████| 489/489 [01:49<00:00,  4.45it/s]


Validation loss: 0.153146133314
Validation Accuracy: 0.955523469453
Validation F1-Score: 0.715634906296


In [293]:
pred_tags

[[u'O',
  u'O',
  u'ORG',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'ORG',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O'],
 [u'O',
  u'O',
  u'ORG',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
 

for lab in b_labels:
    for l in lab:
        print(idx2tag[int(l)])
        

In [384]:
[[ idx2tag[int(l)] for l in lab] for lab in b_labels]

[[u'O',
  u'O',
  u'ORG',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'ORG',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O'],
 [u'O',
  u'O',
  u'ORG',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'ORG',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',